In [1]:
import pandas as pd 
import numpy as np
import os 

In [2]:

df = pd.read_parquet('/mnt/mdrive/athena/CONCEPT.parquet')
df.head()


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,45756805,Pediatric Cardiology,Provider,ABMS,Physician Specialty,S,OMOP4821938,19700101,20991231,None
1,45756804,Pediatric Anesthesiology,Provider,ABMS,Physician Specialty,S,OMOP4821939,19700101,20991231,None
2,45756803,Pathology-Anatomic / Pathology-Clinical,Provider,ABMS,Physician Specialty,S,OMOP4821940,19700101,20991231,None
3,45756802,Pathology - Pediatric,Provider,ABMS,Physician Specialty,S,OMOP4821941,19700101,20991231,None
4,45756801,Pathology - Molecular Genetic,Provider,ABMS,Physician Specialty,S,OMOP4821942,19700101,20991231,None


In [3]:
df['concept_id'].unique

<bound method Series.unique of 0          45756805
1          45756804
2          45756803
3          45756802
4          45756801
             ...   
9794051     2213270
9794052    45889102
9794053     2213271
9794054     2213272
9794055    45889100
Name: concept_id, Length: 9794056, dtype: object>

In [4]:
df['concept_id'].dtype

dtype('O')

In [5]:
df_measurement=pd.read_csv('data/measurement_concept_id.csv')
df_procedure= pd.read_csv('data/procedure_concept_id.csv')

In [6]:
df_measurement.head(2)

,measurement_concept_id,total_occurrences
0,3034426,8971
1,3005872,8671


In [7]:
concept_lookup = df[['concept_id', 'concept_name']].copy()
concept_lookup['concept_id'] = pd.to_numeric(concept_lookup['concept_id'], errors='coerce')

In [11]:
is_unique = concept_lookup['concept_id'].is_unique
print(f"Is concept_id unique? {is_unique}")

Is concept_id unique? True


In [8]:
df_measurement['measurement_concept_id'].dtype

dtype('int64')

In [10]:
df_measurement_final = df_measurement.merge(
    concept_lookup, 
    left_on='measurement_concept_id', 
    right_on='concept_id', 
    how='left'
)
df_measurement_final.rename(columns={'concept_name': 'measurement_concept_name'}, inplace=True)
df_measurement_final.head()

,measurement_concept_id,total_occurrences,concept_id,measurement_concept_name
0,3034426,8971,3034426,Prothrombin time (PT)
1,3005872,8671,3005872,Hemoglobin [Presence] in Blood
2,3050583,8413,3050583,Platelets panel - Blood by Automated count
3,3009425,8232,3009425,Total lung capacity
4,42868738,8193,42868738,Creatinine in peritoneal fluid/Creatinine in s...


In [25]:
df_procedure_final = df_procedure.merge(
    concept_lookup, 
    left_on='procedure_concept_id', 
    right_on='concept_id', 
    how='left'
)
df_procedure_final.rename(columns={'concept_name': 'procedure_concept_name'}, inplace=True)

In [26]:
df_measurement_final.to_csv('data/out/measurement_concept_id.csv')
df_procedure_final.to_csv('data/out/procedure_cocnept_id.csv')

In [4]:
def add_concept_name(df_merged,table_concept_id, df_concept, filename):
    concept_lookup = df_concept[['concept_id', 'concept_name']].copy()
    concept_lookup['concept_id'] = pd.to_numeric(concept_lookup['concept_id'], errors='coerce')

    df_merged = df_merged.merge(
        concept_lookup, 
        left_on=table_concept_id, 
        right_on='concept_id', 
        how='left'
    )
    new_col_name = table_concept_id.replace('_id', '_name')

    df_merged.rename(columns={'concept_name': new_col_name}, inplace=True)
    df_merged.drop(columns=['concept_id'], inplace=True)

    output_dir = os.path.join('data', 'out')
    print(output_dir)
    os.makedirs(output_dir, exist_ok=True)
    full_path = os.path.join(output_dir,f"{filename}.csv")
    df_merged.to_csv(full_path, index=False)
    print(f"Saved successfully to: {full_path}")

    return df_merged


In [34]:
output_dir = os.path.join('data', 'out')
print(output_dir)

data/out


In [7]:
df_procedure_frequency=pd.read_csv('data/procedure_frequency.csv')
df_procedure_frequency=add_concept_name(df_procedure_frequency,'procedure_concept_id',df, 'procedure_frequency')
df_procedure_frequency.head()

data/out
Saved successfully to: data/out/procedure_frequency.csv


,procedure_concept_id,occurrence,distinct_person_count,procedure_concept_name
0,4284104,329,302,Aortocoronary artery bypass graft
1,4203153,288,276,Replacement of mitral valve
2,4012932,170,162,Atrioseptoplasty
3,4095407,81,78,Replacement of aortic valve
4,4304688,79,75,Double valve replacement


In [5]:
df_measurement_frequency=pd.read_csv('data/measurement_frequency.csv')
df_measurement_frequency=add_concept_name(df_measurement_frequency,'measurement_concept_id',df, 'measurement_frequency')
df_measurement_frequency.head()

data/out
Saved successfully to: data/out/measurement_frequency.csv


,measurement_concept_id,frequency,distinct_person_count,measurement_concept_name
0,3034426,8971,1139,Prothrombin time (PT)
1,3005872,8671,1142,Hemoglobin [Presence] in Blood
2,3050583,8413,1141,Platelets panel - Blood by Automated count
3,3009425,8232,1140,Total lung capacity
4,42868738,8193,1141,Creatinine in peritoneal fluid/Creatinine in s...


In [10]:
df_measurement=pd.read_csv('data/measurement_concept_id.csv')
df_measurement=add_concept_name(df_measurement,'measurement_concept_id',df,'measurement_concept')
df_measurement.head()

data/out
Saved successfully to: data/out/measurement_concept.csv


,measurement_concept_id,total_occurrences,measurement_concept_name
0,3034426,8971,Prothrombin time (PT)
1,3005872,8671,Hemoglobin [Presence] in Blood
2,3050583,8413,Platelets panel - Blood by Automated count
3,3009425,8232,Total lung capacity
4,42868738,8193,Creatinine in peritoneal fluid/Creatinine in s...


In [6]:
df_procedure=pd.read_csv('data/procedure_concept_id.csv')
df_procedure=add_concept_name(df_procedure,'procedure_concept_id',df,'procedure_concept')
df_procedure.head()

data/out
Saved successfully to: data/out/procedure_concept.csv


,procedure_concept_id,total_occurrences,procedure_concept_name
0,4284104,329,Aortocoronary artery bypass graft
1,4203153,288,Replacement of mitral valve
2,4012932,170,Atrioseptoplasty
3,4095407,81,Replacement of aortic valve
4,4304688,79,Double valve replacement


In [ ]:
pwd

'/mnt/mdrive/home1/akshat/CVT'

In [ ]:
ls

data/  temp1.ipynb  venv/


In [ ]:
# SELECT * FROM cvts_cdm.death
# SELECT 
#     ROUND(
#         (SELECT COUNT(*) FROM cvts_cdm.death) * 100.0 /
#         (SELECT COUNT(*) FROM cvts_cdm.person),
#         2
#     ) AS mortality_percentage;

# SELECT 
#     person_id,
#     measurement_concept_id,
#     COUNT(*) as frequency
# FROM cvts_cdm.measurement
# GROUP BY person_id, measurement_concept_id
# ORDER BY person_id, frequency DESC;


# SELECT  
#     measurement_concept_id,
#     COUNT(*) as frequency
# FROM cvts_cdm.measurement
# GROUP BY measurement_concept_id
# ORDER BY  frequency DESC;



# SELECT 
#     gender,
#     total_persons,
#     total_deaths,
#     ROUND((total_deaths * 100.0) / NULLIF(total_persons, 0), 2) AS mortality_percentage
# FROM (
#     -- ALL
#     SELECT 
#         'ALL' AS gender,
#         COUNT(*) AS total_persons,
#         (SELECT COUNT(*) FROM cvts_cdm.death) AS total_deaths
#     FROM cvts_cdm.person

#     UNION ALL

#     -- MALE
#     SELECT 
#         'MALE' AS gender,
#         COUNT(*) AS total_persons,
#         (SELECT COUNT(*) 
#          FROM cvts_cdm.death d
#          JOIN cvts_cdm.person p ON p.person_id = d.person_id
#          WHERE p.gender_concept_id = 8507) AS total_deaths
#     FROM cvts_cdm.person
#     WHERE gender_concept_id = 8507

#     UNION ALL

#     -- FEMALE
#     SELECT 
#         'FEMALE' AS gender,
#         COUNT(*) AS total_persons,
#         (SELECT COUNT(*) 
#          FROM cvts_cdm.death d
#          JOIN cvts_cdm.person p ON p.person_id = d.person_id
#          WHERE p.gender_concept_id = 8532) AS total_deaths
#     FROM cvts_cdm.person
#     WHERE gender_concept_id = 8532
# ) AS sub;
